# Drug screening dataset

In [1]:
import gilda
import pickle
import synapseclient
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
from rpy2 import robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

INFO: [2020-11-13 18:03:47] /Users/ben/Dropbox/postdoc/darpa/src/deft/adeft/recognize.py - OneShotRecognizer not available. Extension module for AlignmentBasedScorer is missing
INFO: [2020-11-13 18:03:47] keyring.backend - Loading SecretService
INFO: [2020-11-13 18:03:47] keyring.backend - Loading kwallet
INFO: [2020-11-13 18:03:47] keyring.backend - Loading macOS
INFO: [2020-11-13 18:03:47] keyring.backend - Loading windows


In [2]:
syn = synapseclient.Synapse()
syn.login()

Welcome, bengyori!



INFO: [2020-11-13 18:03:49] synapseclient_default - Welcome, bengyori!



## Process drug response data

### Download data as pandas DataFrame

In [3]:
drug_data_path = syn.get("syn20684161").path
drug_data = pd.read_csv(drug_data_path, low_memory=False)
pd.DataFrame.head(drug_data)

,model_name,model_type,cellosaurus_id,organism_name,disease_name,disease_efo_id,symptom_name,symptom_efo_id,experiment_synapse_id,study_synapse_id,funder,drug_name,DT_explorer_internal_id,dosage_unit,drug_screen_id,dosage,response_type,response,response_unit
0,N10,cell line,NaN,human,no disease,NaN,no symptom,NaN,syn11373153.1,syn5610425,CTF,GPHR-00000018,159129,uM,1,"[0.0457246089549396,100]",AUC_Simpson,35.508784,NaN
1,N5,cell line,NaN,human,NF1,NaN,no symptom,NaN,syn11373157.1,syn5610425,CTF,GPHR-00000291,194936,uM,10,"[0.0457246089549396,100]",AUC_Simpson,4.018050,NaN
2,N5,cell line,NaN,human,NF1,NaN,no symptom,NaN,syn11373207.1,syn5610425,CTF,GPHR-00101658,313081,uM,100,"[0.0457246089549396,100]",AUC_Simpson,23.566992,NaN
3,N5,cell line,NaN,human,NF1,NaN,no symptom,NaN,syn11373765.1,syn5610425,CTF,GPHR-00224355,313317,uM,1000,"[0.0457246089549396,100]",AUC_Simpson,22.773713,NaN
4,Syn5,cell line,NaN,human,NF2,NaN,no symptom,NaN,syn12293953.1,syn2343195,CTF,NCGC00241342-02,150042,uM,10000,"[0.00078041474654,46.082949308]",AUC_Simpson,45.188508,NaN


### Filter data
Here we filter the data to drugs that have an IC50 of at most 10.

In [28]:
IC50_CUTOFF = 10
pnf = ["ipNF05.5", "ipNF06.2A", "ipNF95.11b C/T", "ipnNF95.11C", "ipNF95.6", "ipNF05.5 (mixed clone)", "ipNF95.11b C"]

drug_data_pnf = (drug_data
      .query('response_type == "IC50_abs"')
      .query('model_name == @pnf')
      .groupby('drug_screen_id').filter(lambda x: len(x)==1)
      .assign(response = lambda x: [50 if a >= 50 else a for a in x['response']]))
drug_data_pnf_positive = drug_data_pnf[drug_data_pnf.response < IC50_CUTOFF]
positive_drug_ids = sorted(list(drug_data_pnf_positive.DT_explorer_internal_id.unique()))
len(positive_drug_ids)

1072

## Process drug-target data

In [7]:
targetspath = syn.get('syn17091507')
readRDS = robjects.r['readRDS']
targets = readRDS(targetspath.path)

In [9]:
targets.head()

,internal_id,hugo_gene,n_quantitative,mean_pchembl,cv,sd,IC50_nM,AC50_nM,EC50_nM,Potency_nM,Ki_nM,Kd_nM,n_qualitative,std_name,total_n,confidence,pchembl_d,pchembl_t,known_selectivity_index
1,3,HTR7,1,8.01,NaN,NaN,NaN,NaN,NaN,NaN,9.7,NaN,-2147483648,CHEMBL2413451,1,-0.229,8.01,10322.90,1.0
2,4,CHRNA4,1,7.54,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,-2147483648,CHEMBL204871,1,-0.229,15.08,4501.97,0.5
3,4,CHRNB2,1,7.54,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,-2147483648,CHEMBL204871,1,-0.229,15.08,4194.75,0.5
4,5,GSK3A,2,7.76,5.285076,0.410122,34.0,NaN,NaN,NaN,9.0,NaN,-2147483648,CHEMBL3582401,2,0.393,7.76,2034.63,1.0
5,6,FAAH,1,9.30,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,-2147483648,CHEMBL2386554,1,-0.229,9.30,6375.76,1.0


Here we get the names of the drugs that passed the IC50 filter above

In [29]:
drug_names = \
    sorted(targets.std_name[
        targets.internal_id.isin(set(positive_drug_ids))].unique())

In [30]:
len(drug_names)

661

Now we need to find groundings for these drug names

In [32]:
from indra.ontology.bio import bio_ontology
drug_groundings = {}
for drug_name in drug_names:
    grounding = None
    # First handle names that are actually CHEMBL IDs
    if drug_name.startswith('CHEMBL'):
        grounding = ('CHEMBL', drug_name)
    # Next, try looking up some of these by standard name
    # This is needed because we have CHEMBL standard names in INDRA
    # that Gilda doesn't have e.g., TOZASERTIB
    else:
        for ns in ['CHEMBL', 'CHEBI']:
            ns_id_from_standard_name = bio_ontology.get_id_from_name(ns, drug_name)
            if ns_id_from_standard_name:
                grounding = ns_id_from_standard_name
                break
            else:
                ns_id_from_standard_name = bio_ontology.get_id_from_name(ns, drug_name.upper())
                if ns_id_from_standard_name:
                    grounding = ns_id_from_standard_name
                    break
        # Finally, try Gilda to see if the name can be grounded
        else:
            matches = gilda.ground(drug_name)
            if matches:
                grounding = (matches[0].term.db, matches[0].term.id)
    drug_groundings[drug_name] = grounding if grounding else None

In [34]:
'Ungrounded drugs: %s' % len([k for k, v in drug_groundings.items() if not v])

'Ungrounded drugs: 50'

Now we turn the drugs names and groundings into standardized INDRA Agents

In [35]:
from indra.statements import Agent, Inhibition
from indra.ontology.standardize import standardize_name_db_refs
drug_agents = []
for drug_name, grounding in drug_groundings.items():
    if not grounding:
        agent = Agent(drug_name)
        drug_agents.append(agent)
        continue
    standard_name, db_refs = standardize_name_db_refs({grounding[0]: grounding[1]})
    agent_name = standard_name if standard_name else drug_name
    agent = Agent(agent_name, db_refs=db_refs)
    drug_agents.append(agent)

We also have to make a standardized INDRA Agent representing cell proliferation

In [36]:
match = gilda.ground('cell proliferation')[0]
standard_name, db_refs = standardize_name_db_refs({match.term.db: match.term.id})
cell_proliferation = Agent(standard_name, db_refs=db_refs)

We can now create a set of Inhibition statements between each drug and cell proliferation, and repersent these as EMMAA StatementCheckingTests.

In [19]:
from emmaa.model_tests import StatementCheckingTest
test_statements = [StatementCheckingTest(Inhibition(drug_agent, cell_proliferation))
                   for drug_agent in drug_agents]

INFO: [2020-11-13 18:04:38] covid_19.preprocess - Latest data release is 2020-11-12
INFO: [2020-11-13 18:04:38] pybel.config - no configuration found, using default sqlite connection sqlite:////Users/ben/.pybel/pybel_0.14.0_cache.db
INFO: [2020-11-13 18:04:50] indra_reading.readers.mti - Got MTI_EMAIL from environment
INFO: [2020-11-13 18:04:50] indra_reading.readers.mti - Got MTI_USERNAME from environment
INFO: [2020-11-13 18:04:50] indra_reading.readers.mti - Got MTI_PASSWORD from SSM
INFO: [2020-11-13 18:04:50] indra_reading.readers.mti - Got MTI_JARS_PATH from environment
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-

INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: 

INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: 

INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: 

INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: 

INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: None
INFO: [2020-11-13 18:04:50] emmaa.model_tests - Test configs: 

In [20]:
import pickle
with open('drug_screening_test_stmts.pkl', 'wb') as fh:
    pickle.dump(test_statements, fh)

Given the list of drugs we identified, we can also get all INDRA Statements that report one of these drugs as inhibiting or binding to a protein target.

In [37]:
from covid_19.get_drug_statements import get_drug_groundings, get_drug_statements
drug_groundings = get_drug_groundings(drug_agents)
drug_stmts = get_drug_statements(drug_groundings)

INFO: [2020-11-13 19:14:58] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?subject=CHEBI:95067@CHEBI&offset=0
INFO: [2020-11-13 19:14:58] indra.sources.indra_db_rest.util - params: {'ev_limit': 100, 'best_first': True, 'api_key': '[api-key]'}


No grounding for (3-{3-[[2-Chloro-3-(Trifluoromethyl)Benzyl](2,2-Diphenylethyl)Amino]Propoxy}Phenyl)Acetic Acid() ({})
No grounding for (3e)-6'-Bromo-2,3'-Biindole-2',3(1h,1'h)-Dione 3-Oxime() ({})
No grounding for 2-({5-CHLORO-2-[(2-METHOXY-4-MORPHOLIN-4-YLPHENYL)AMINO]PYRIMIDIN-4-YL}AMINO)-N-METHYLBENZAMIDE() ({})
No grounding for 2-{[(1R,2S)-2-aminocyclohexyl]amino}-4-[(3-methylphenyl)amino]pyrimidine-5-carboxamide() ({})
No grounding for 3-[(3-(2-CARBOXYETHYL)-4-METHYLPYRROL-2-YL)METHYLENE]-2-INDOLINONE() ({})
No grounding for 4-(4-chlorobenzyl)-1-(7H-pyrrolo[2,3-d]pyrimidin-4-yl)piperidin-4-aminium() ({})
No grounding for 4-[(3S)-1-AZABICYCLO[2.2.2]OCT-3-YLAMINO]-3-(1H-BENZIMIDAZOL-2-YL)-6-CHLOROQUINOLIN-2(1H)-ONE() ({})
No grounding for 4-[3-(4-chlorophenyl)-2,1-benzisoxazol-5-yl]pyrimidin-2-amine() ({})
No grounding for 619C89() ({})
No grounding for 7-[4-(Dimethylamino)Phenyl]-N-Hydroxy-4,6-Dimethyl-7-Oxo-2,4-Heptadienamide() ({})
No grounding for AM-630() ({})
No grounding for

INFO: [2020-11-13 19:14:59] indra.tools.assemble_corpus - Filtering 28 statements for type Inhibition...
INFO: [2020-11-13 19:14:59] indra.tools.assemble_corpus - 7 statements after filter...
INFO: [2020-11-13 19:14:59] indra.tools.assemble_corpus - Filtering 28 statements for type Complex...
INFO: [2020-11-13 19:14:59] indra.tools.assemble_corpus - 1 statements after filter...
INFO: [2020-11-13 19:14:59] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?subject=CHEBI:7781@CHEBI&offset=0
INFO: [2020-11-13 19:14:59] indra.sources.indra_db_rest.util - params: {'ev_limit': 100, 'best_first': True, 'api_key': '[api-key]'}


Searching for CHEBI:7781@CHEBI


INFO: [2020-11-13 19:15:00] indra.tools.assemble_corpus - Filtering 124 statements for type Inhibition...
INFO: [2020-11-13 19:15:00] indra.tools.assemble_corpus - 46 statements after filter...
INFO: [2020-11-13 19:15:00] indra.tools.assemble_corpus - Filtering 124 statements for type Complex...
INFO: [2020-11-13 19:15:00] indra.tools.assemble_corpus - 4 statements after filter...
INFO: [2020-11-13 19:15:00] indra.sources.indra_db_rest.util - query: https://db.indra.bio/statements/from_agents?subject=CHEBI:10429@CHEBI&offset=0
INFO: [2020-11-13 19:15:00] indra.sources.indra_db_rest.util - params: {'ev_limit': 100, 'best_first': True, 'api_key': '[api-key]'}


Searching for CHEBI:10429@CHEBI


KeyboardInterrupt: 

In [26]:
with open('drug_statements.pkl', 'wb') as fh:
    pickle.dump(drug_stmts, fh)

## Notebook for old dataset

In [22]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()


In [23]:
targetspath = syn.get('syn17091507')
readRDS = robjects.r['readRDS']

targets = readRDS(targetspath.path)

In [24]:

targets_filt = (targets
                .query('mean_pchembl > 6')
                .filter(["internal_id", "hugo_gene", "std_name"])
                .drop_duplicates())
    
pd.DataFrame.head(targets_filt)

,internal_id,hugo_gene,std_name
1,3,HTR7,CHEMBL2413451
2,4,CHRNA4,CHEMBL204871
3,4,CHRNB2,CHEMBL204871
4,5,GSK3A,CHEMBL3582401
5,6,FAAH,CHEMBL2386554


In [25]:
drug_data_path = syn.get("syn17462699").path
drug_data = pd.read_csv(drug_data_path, low_memory=False)
                       
pd.DataFrame.head(drug_data)

SynapseHTTPError: 403 Client Error: 
You do not have READ permission for the requested entity, syn17462699.